## Imports

In [50]:
import os
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
import glob

## Loading docs

In [51]:
pdf_path_1="legal_docs/contrat_travail_1.pdf"
pdf_path_2="legal_docs/convention-premium-portage-1.pdf"
txt_path_1="legal_docs/contrat_freelance.txt"
txt_path_2="legal_docs/conditions_generales.txt"

def load_documents():
    """Load all PDF and text documents"""
    documents = []
    
    # Load PDF document
    pdf_files = [pdf_path_1, pdf_path_2]  # Update paths
    for pdf_path in pdf_files:
        try:
            pdf_loader = PyPDFLoader(pdf_path)  # Update path
            pdf_docs = pdf_loader.load()
            documents.extend(pdf_docs)
            print(f"Loaded PDF with {len(pdf_docs)} pages")
        except Exception as e:
            print(f"Error loading PDF: {e}")
    
    # Load text documents
    text_files = [txt_path_1, txt_path_2]  # Update paths

    for text_file in text_files:
        try:
            if os.path.exists(text_file):
                text_loader = TextLoader(text_file)
                text_docs = text_loader.load()
                documents.extend(text_docs)
                print(f"Loaded text file: {text_file}")
        except Exception as e:
            print(f"Error loading {text_file}: {e}")
    
    print(f"Total documents loaded: {len(documents)}")
    return documents

In [52]:
documents = load_documents()

Loaded PDF with 4 pages
Loaded PDF with 23 pages
Loaded text file: legal_docs/contrat_freelance.txt
Loaded text file: legal_docs/conditions_generales.txt
Total documents loaded: 29


In [53]:
print(f"First document content preview: {type(documents[0])}")
print(documents[0].metadata)  

First document content preview: <class 'langchain_core.documents.base.Document'>
{'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-10-29T16:36:47+01:00', 'author': '', 'moddate': '2025-10-29T16:36:47+01:00', 'title': 'Microsoft Word - Document3', 'source': 'legal_docs/contrat_travail_1.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}


## Splitting docs (Chunking)

In [54]:
def split_documents(documents):
    """Split documents into chunks using recursive text splitting"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,           # Optimal for most use cases
        chunk_overlap=200,         # Maintains context between chunks
        length_function=len,
        is_separator_regex=False,
        separators=["\n\n", "\n", ". ", " ", ""]  # Default separators
    )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")
    return chunks

In [55]:
chunks = split_documents(documents)

Split into 105 chunks


In [56]:
print(f"First document content preview: {chunks[0].page_content}")

First document content preview: CONTRAT DE TRAVAIL À DURÉE INDÉTERMINÉE (CDI) 
 
Entre les soussignés : 
 
La société TECHCORP SAS, au capital de 500 000 euros, immatriculée au RCS de Paris 
sous le numéro 123 456 789, dont le siège social est situé au 15 Avenue des Champs-
Élysées, 75008 Paris, représentée par Monsieur Jean Dupont, agissant en qualité de 
Directeur Général, ci-après dénommée « l'Employeur », 
 
D'une part, 
 
Et : 
 
Madame Marie Martin, née le 12 mars 1990 à Lyon, de nationalité française, demeurant 
au 45 rue de la République, 69002 Lyon, ci-après dénommée « le Salarié », 
 
D'autre part, 
 
Il a été convenu et arrêté ce qui suit : 
 
ARTICLE 1 - ENGAGEMENT 
 
L'Employeur engage le Salarié qui accepte, aux clauses et conditions ci-après 
énoncées, en qualité d'Ingénieur Data Scientist, position cadre, niveau III de la 
convention collective applicable. 
 
ARTICLE 2 - DURÉE DU CONTRAT 
 
Le présent contrat est conclu pour une durée indéterminée et prendra eƯet le 1er

## Creating the vectorstore & Storing

In [58]:
#function to delete existing chroma db if exists (delete completely for all memories)
def delete_existing_vectorstore(persist_directory="./legal_chroma_db_"):
    """Delete existing ChromaDB vectorstore directory"""
    if os.path.exists(persist_directory):
        files = glob.glob(os.path.join(persist_directory, '*'))
        for f in files:
            os.remove(f)
        os.rmdir(persist_directory)
        print(f"Deleted existing vectorstore at {persist_directory}")
    else:
        print(f"No existing vectorstore found at {persist_directory}")

In [ ]:
delete_existing_vectorstore()

In [60]:
def create_vectorstore(chunks, persist_directory="./_legal_chroma_db_"):
    """Create and persist ChromaDB vectorstore"""
    # Initialize embeddings - using OpenAI for best results
    embeddings = OllamaEmbeddings(
        model="embeddinggemma",
    )
    
    # Create vectorstore
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    
    # Persist to disk
    vectorstore.persist()
    print(f"Vectorstore created and persisted to {persist_directory}")
    return vectorstore

In [61]:
vectorstore = create_vectorstore(chunks)

Vectorstore created and persisted to ./_legal_chroma_db_


## Loading the vectorstore to use it 

In [ ]:
def load_existing_vectorstore(persist_directory="./legal_chroma_db_"):
    """Load an existing vectorstore"""
    embeddings = OllamaEmbeddings(
        model="embeddinggemma",
    )
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
    return vectorstore

## Testing the code

In [62]:
test_query = "c'est quoi le capital de techcorp sas"
results = vectorstore.similarity_search(test_query, k=10)

In [63]:
results

[Document(metadata={'moddate': '2025-10-29T16:36:47+01:00', 'page_label': '1', 'producer': 'Microsoft: Print To PDF', 'source': 'legal_docs/contrat_travail_1.pdf', 'total_pages': 4, 'creator': 'PyPDF', 'creationdate': '2025-10-29T16:36:47+01:00', 'page': 0, 'author': '', 'title': 'Microsoft Word - Document3'}, page_content="CONTRAT DE TRAVAIL À DURÉE INDÉTERMINÉE (CDI) \n \nEntre les soussignés : \n \nLa société TECHCORP SAS, au capital de 500 000 euros, immatriculée au RCS de Paris \nsous le numéro 123 456 789, dont le siège social est situé au 15 Avenue des Champs-\nÉlysées, 75008 Paris, représentée par Monsieur Jean Dupont, agissant en qualité de \nDirecteur Général, ci-après dénommée « l'Employeur », \n \nD'une part, \n \nEt : \n \nMadame Marie Martin, née le 12 mars 1990 à Lyon, de nationalité française, demeurant \nau 45 rue de la République, 69002 Lyon, ci-après dénommée « le Salarié », \n \nD'autre part, \n \nIl a été convenu et arrêté ce qui suit : \n \nARTICLE 1 - ENGAGEMENT 

In [47]:
print(type(vectorstore))

<class 'langchain_community.vectorstores.chroma.Chroma'>
